In [10]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch


In [2]:
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')


/Users/damodargupta/anaconda3/envs/latest_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model.safetensors: 100%|██████████| 440M/440M [06:25<00:00, 1.14MB/s] 
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
sentences = ["Your negotiation text here.", "Another sentence."]
tokenized_texts = [tokenizer.tokenize(sentence) for sentence in sentences]

In [5]:
tokenized_texts

[['your', 'negotiation', 'text', 'here', '.'], ['another', 'sentence', '.']]

In [6]:
training_data = {
    "The iPhone 13 Pro is priced at $999, but we have a limited-time offer with a 20% discount.": {
        "Product": "iPhone 13 Pro",
        "Price": "$999",
        "Discount": "20%"
    },
    "The Galaxy Watch 4 comes with a 2-year warranty and free shipping.": {
        "Product": "Galaxy Watch 4",
        "Warranty": "2 years",
        "Shipping": "Free"
    },
    "Our new smart TV model features a 55-inch 4K OLED display and built-in streaming apps.": {
        "Product": "Smart TV",
        "Display": "55-inch 4K OLED",
        "Features": "Built-in streaming apps"
    },
    "This laptop has an Intel Core i7 processor, 16GB RAM, and a 512GB SSD for fast performance.": {
        "Product": "Laptop",
        "Processor": "Intel Core i7",
        "RAM": "16GB",
        "Storage": "512GB SSD"
    },
    "Our vacation package includes a 7-day stay at a 5-star beachfront resort and daily breakfast.": {
        "Package": "Vacation package",
        "Duration": "7 days",
        "Accommodation": "5-star beachfront resort",
        "Inclusions": "Daily breakfast"
    },
    "The annual subscription to our streaming service offers unlimited access to a vast library of movies and TV shows.": {
        "Service": "Streaming service",
        "Subscription": "Annual",
        "Content": "Movies and TV shows"
    },
    "Our car rental service provides insurance coverage and a fuel-efficient vehicle for your trip.": {
        "Service": "Car rental",
        "Insurance": "Coverage included",
        "Vehicle": "Fuel-efficient"
    },
    "The mortgage rate for this property is 3.5% with a 30-year term.": {
        "Property": "Real estate property",
        "Interest Rate": "3.5%",
        "Loan Term": "30 years"
    },
    "Our fitness membership includes access to all gym facilities, group classes, and personal training sessions.": {
        "Membership": "Fitness membership",
        "Access": "Gym facilities, group classes, personal training"
    },
    "The credit card offers a cashback reward of 2% on all purchases and no annual fee.": {
        "Credit Card": "Credit card",
        "Rewards": "2% cashback",
        "Fees": "No annual fee"
    },
    "The new Samsung Galaxy S22 Ultra, with its 108MP camera, is available for pre-order at $1,199, and you can trade in your old device for an extra $300 off.": {
        "Product": "Samsung Galaxy S22 Ultra",
        "Camera": "108MP",
        "Price": "$1,199",
        "Discount": "$300 off (with trade-in)"
    },
    "Our 65-inch 8K OLED TV, featuring Dolby Atmos sound and smart home integration, is on sale for $1,999, down from the original price of $2,499.": {
        "Product": "65-inch 8K OLED TV",
        "Features": "Dolby Atmos sound, smart home integration",
        "Original Price": "$2,499",
        "Sale Price": "$1,999"
    },
    "Experience a luxury cruise with our 10-day Mediterranean package on the Grand Oceanic cruise liner, inclusive of all meals and excursions, starting at $3,499 per person.": {
        "Package": "10-day Mediterranean cruise",
        "Cruise Liner": "Grand Oceanic",
        "Inclusions": "All meals, excursions",
        "Price": "$3,499 per person"
    },
    "Upgrade your home office setup with the latest MacBook Pro, equipped with an M2 chip, 16GB RAM, and a 512GB SSD, available for $1,799.": {
        "Product": "MacBook Pro",
        "Processor": "M2 chip",
        "RAM": "16GB",
        "Storage": "512GB SSD",
        "Price": "$1,799"
    },
    "Join our premium gym membership program, which includes access to personal trainers, nutrition coaching, and unlimited classes for $99 per month, with a 20% discount on the first three months.": {
        "Membership": "Premium gym membership",
        "Inclusions": "Personal trainers, nutrition coaching, unlimited classes",
        "Monthly Price": "$99",
        "Discount": "20% off (first three months)",
        "time period" : "3 Months"
    },
    "Get a home loan with a fixed interest rate of 3.25% for a 15-year term, and we offer a $5,000 credit towards closing costs for qualified applicants.": {
        "Loan Type": "Home loan",
        "Interest Rate": "3.25% (fixed)",
        "Loan Term": "15 years",
        "Closing Costs": "$5,000 credit (qualified applicants)"
    },
    "The annual membership for our streaming service provides 4K streaming, offline downloads, and access to exclusive content for $12.99 per month, billed annually.": {
        "Service": "Streaming service",
        "Features": "4K streaming, offline downloads, exclusive content",
        "Monthly Price": "$12.99",
        "Billing": "Annually"
    },
    "Fly with our airline and enjoy complimentary in-flight meals, a 30kg baggage allowance, and free Wi-Fi on all international flights.": {
        "Airline": "Our airline",
        "Inclusions": "Complimentary meals, 30kg baggage allowance, free Wi-Fi (international flights)"
    },
    "Our credit card offers a 1.5% cashback reward on all purchases, no foreign transaction fees, and a $200 bonus for new cardholders who spend $1,500 in the first 3 months.": {
        "Credit Card": "Credit card",
        "Rewards": "1.5% cashback",
        "Fees": "No foreign transaction fees",
        "Bonus": "$200 (for new cardholders spending $1,500 in the first 3 months)"
    }
}

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

train_dataset = training_data

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

MODEL_NAME = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

Using device: cpu


Trying to resume download...














model-00001-of-00002.safetensors:   6%|▋         | 315M/5.00G [02:07<1:19:08, 986kB/s] 
model-00001-of-00002.safetensors:   4%|▍         | 189M/5.00G [06:06<2:35:40, 515kB/s]


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./phi2_fine_tuned",
    per_device_train_batch_size=8,  # Adjust batch size as needed
    num_train_epochs=3,             # Adjust the number of epochs
    evaluation_strategy="steps",
    eval_steps=500,                 # Evaluate the model every 500 steps
    save_steps=500,                 # Save checkpoints every 500 steps
    save_total_limit=2,             # Limit the number of checkpoints to save
)

# Define a Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./phi2_fine_tuned")

# Optionally, save the tokenizer as well
tokenizer.save_pretrained("./phi2_fine_tuned_tokenizer")
